In [195]:
from IPython.display import display, HTML
from dataclasses import dataclass

@dataclass
class HtmlPrinter:
    font_size:int = 15
    line_height:float = 1.5
    white_space: str = "pre-wrap"
    width: int = 1000
    text_align: str = "justify"
    font_family: str = "HelveticaNeue-Light"

    @property
    def open_tag(self):
        return f'<div style="font-size: {self.font_size}px; line-height:{self.line_height}em; white-space: {self.white_space}; width:{self.width}px; text-align:{self.text_align}; font-family:{self.font_family}">'
    
    @property
    def close_tag(self):
        return "</div>"
    
    def __call__(self, content):
        display(HTML(self.open_tag + content + self.close_tag))

printer = HtmlPrinter()

## Model object - LLM

In [1]:
from langchain.llms import OpenAI

# you can pass any parmaeters of openai Chat APi to LLM constructor
llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0.9)

prompt = "Hello there, are you chatGPT?"

# llm is a callable
completion = llm(prompt)

print(completion)



Hello, yes! I am chatGPT, an AI-powered chatbot. How can I assist you today?


## Prompt Object

In [31]:
from langchain.prompts import PromptTemplate

# crete an PromptTemplate object
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

prompt

PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='What is a good name for a company that makes {product}?', template_format='f-string', validate_template=True)

In [32]:
# just a simple string formatting.
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


## Combine LLM and prompt

In [4]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# an langchain takes in a LLM model and a prompt template
chain = LLMChain(llm=llm, prompt=prompt)

# you can format template here
# and you got an answer from OPENAI
chain.run("colorful socks")

'\n\nRainbow Socks Co.'

## Conversational Memory

One of the simpler forms of memory occurs in chatbots, where they remember previous conversations. There are a few different ways to accomplish this:     

- Buffer: This is just passing in the past N interactions in as context. N can be chosen based on a fixed number, the length of the interactions, or other!
- Summary: This involves summarizing previous conversations and passing that summary in, instead of the raw dialogue itself. Compared to Buffer, this compresses information: meaning it is more lossy, but also less likely to run into context length limits.
- Combination: A combination of the above two approaches, where you compute a summary but also pass in some previous interactions directly!

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

**ConversationBufferMemory**

In [9]:
# return the whole hsitory as a string
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

# return chat history as a list of objects, which looks like
"""
{
    'history': [
    HumanMessage(content='Hi there!', additional_kwargs={}),
    AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}),
    HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}),
    AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={}),
    HumanMessage(content='Tell me about yourself.', additional_kwargs={})
    ]
}
"""

# during chatting, the list history is automatically built.
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [25]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}),
  AIMessage(content='whats up', additional_kwargs={})]}

In [33]:
llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0)
memory = ConversationBufferMemory(return_messages = True)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory= memory,
)

In [34]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [35]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={})]
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?"

In [36]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}), HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}), AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={})]
Human: Tell me about yourself.
AI:

> Finished chain.


"Sure! I am an AI language model designed to assist humans with various tasks and provide helpful information. I was created by a team of developers who used advanced machine learning algorithms to train me on a vast amount of data. I am constantly learning and improving my abilities to better serve my users. Is there anything else you'd like to know about me?"

In [37]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi there!', additional_kwargs={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}),
  HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}),
  AIMessage(content="That's great to hear! I'm always happy to chat with humans. Is there anything specific you'd like to talk about or ask me?", additional_kwargs={}),
  HumanMessage(content='Tell me about yourself.', additional_kwargs={}),
  AIMessage(content="Sure! I am an AI language model designed to assist humans with various tasks and provide helpful information. I was created by a team of developers who used advanced machine learning algorithms to train me on a vast amount of data. I am constantly learning and improving my abilities to better serve my users. Is there anything else you'd like to know about me?", additional_kwargs={})]}

**ConversationBufferWindowMemory**

In [40]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: haha\nAI: that's it"}

In [41]:
memory = ConversationBufferWindowMemory( k=2)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: not much you\nAI: not much\nHuman: haha\nAI: that's it"}

In [42]:
memory = ConversationBufferWindowMemory( k=3)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})
memory.save_context({"input": "haha"}, {"output":"that's it"})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: whats up\nHuman: not much you\nAI: not much\nHuman: haha\nAI: that's it"}

**ConversationEntityMemory**  
   
Add entity information to the conversation memory.    

This is useful for tasks like dialogue state tracking, where you want to keep track the opinion and concept of entities.   

这种方式会比较浪费钱。

In [51]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

# note this memory requires a llm object
memory = ConversationEntityMemory(llm=llm)

_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)


memory.save_context(
    _input,
    {"ouput": " That sounds like a great project! What kind of project are they working on?"}
)


memory.load_memory_variables({"input": 'who is Deven'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam.',
  'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [50]:
memory.load_memory_variables({"input": 'who is Deven'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam.',
  'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [197]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
printer(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

In [202]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
llm = OpenAI(model_name = "gpt-3.5-turbo",temperature=0)

conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [203]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'That sounds exciting! What kind of project are they working on for the hackathon? Do you have any more information about it?'

In [204]:
conversation.memory.store

{'Deven': 'Deven is currently working on a hackathon project with Sam.',
 'Sam': 'Sam is currently working on a hackathon project with Deven.',
 'hackathon': 'Deven and Sam are currently participating in a hackathon project.'}

In [205]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>
).


'Ah, I see. So Deven and Sam are working on adding more complex memory structures to Langchain for the hackathon project. That sounds like a challenging task, but also very interesting. Can you tell me more about Langchain and what it is used for?'

In [206]:
conversation.memory.store

{'Deven': 'Deven is working on adding more complex memory structures to Langchain for the hackathon project.',
 'Sam': 'Sam is working with Deven to add more complex memory structures to Langchain for their hackathon project.',
 'hackathon': 'Deven and Sam are currently participating in a hackathon project.',
 'Langchain': 'Langchain is a project that Deven and Sam are working on for the hackathon, where they are trying to add more complex memory structures.'}

In [207]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Based on the context you provided, I know that Deven and Sam are working together on a hackathon project to add more complex memory structures to Langchain. Unfortunately, I don't have any additional information about them beyond that. Is there anything specific you would like to know?"

## ConversationKGMemory

Actually knowledge graph memory, 反正挺傻逼的，维护好多三元组。

In [210]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"ouput": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"ouput": "okay"})

memory.load_memory_variables({"input": 'who is sam'})

{'history': 'On Sam: Sam is friend.'}

In [212]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"ouput": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"ouput": "okay"})
memory.load_memory_variables({"input": 'who is sam'})

{'history': [SystemMessage(content='On Sam: Sam is friend.', additional_kwargs={})]}

In [213]:
memory.get_current_entities("what's Sams favorite color?")

['Sam']

In [214]:
memory.get_knowledge_triplets("her favorite color is red")

[KnowledgeTriple(subject='Sam', predicate='favorite color', object_='red')]

In [220]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

memory = ConversationKGMemory(llm=llm, return_messages=True)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=memory
)

In [221]:
conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

[]

Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently working on a project to help people better understand the world around them. I'm gathering data and analyzing it to create a comprehensive picture of the environment."

In [222]:
conversation_with_kg.predict(input="My name is James and I'm helping Will. He's an engineer.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

[]

Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


" Hi James, it's nice to meet you. I'm an AI and I understand you're helping Will, the engineer. What kind of engineering does he do?"

In [223]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

[SystemMessage(content='On Will: Will is an engineer.', additional_kwargs={})]

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


' Will is an engineer.'

In [227]:
from langchain.memory import ConversationSummaryMemory

# note that there is a llm object required to create memroy
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "Glad to meet you, we are about to talk about math today?"}, {"ouput": "Sure, what math topic to talk about?"})
memory.save_context({"input": "We are going to talk about kl divergence."}, {"ouput": "KL divergence is a measure of how one probability distribution is different from a second, reference probability distribution."})
memory.save_context({"input": "Next we are going to talk about sin function"}, {"ouput": "Sin function is a function that takes an angle and returns a number."})
memory.load_memory_variables({})

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>
).


{'history': '\nThe human and AI are about to talk about math, and the AI asks what math topic they should discuss. The human suggests they discuss KL divergence, which the AI explains is a measure of how one probability distribution is different from a second, reference probability distribution. The human then suggests they talk about sin function, which the AI explains is a function that takes an angle and returns a number.'}

In [228]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"